# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Week 8 - Notebook 02: Tree-Based Models Complete Guide
**Instructor:** Amir Charkhi | **Goal:** Master decision trees and ensemble methods

> From single trees to powerful forests

## 📚 What You'll Learn Today

**Tree-Based Models Covered:**
1. ✅ Decision Trees (Classification & Regression)
2. ✅ Random Forest (Bagging)
3. ✅ Gradient Boosting (Boosting)
4. ✅ XGBoost (Advanced Boosting)
5. ✅ Feature Importance
6. ✅ Hyperparameter Tuning

**Why Tree Models?**
- Handle non-linear relationships
- No feature scaling needed
- Automatic feature interactions
- Easy to interpret (single tree)
- Often win Kaggle competitions!

**Time**: 90 minutes | **Prerequisites**: Week 7 & Linear Models

---

In [ ]:
# Essential imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("🌳 WEEK 8: TREE-BASED MODELS")
print("✅ Setup complete!")

In [ ]:
# ML imports
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    mean_squared_error, r2_score
)
from sklearn.datasets import make_classification, load_diabetes

print("✅ Tree libraries loaded!")

---

## Part 1: Understanding Decision Trees

**How it works:** Series of yes/no questions

**Example:**
```
Is age > 30?
├─ Yes: Is income > 50k?
│  ├─ Yes: Approve loan ✅
│  └─ No:  Deny loan ❌
└─ No:  Deny loan ❌
```

### 1.1: Create Simple Dataset

In [ ]:
# Generate classification data
np.random.seed(42)
X, y = make_classification(
    n_samples=500,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_clusters_per_class=1,
    random_state=42
)

print(f"Dataset: {len(X)} samples")
print(f"Features: {X.shape[1]}")
print(f"Classes: {np.unique(y)}")
print(f"Class balance: {(y==1).mean():.1%} positive")

### 1.2: Visualize the Data

In [ ]:
# Plot the two classes
plt.figure(figsize=(10, 6))
colors = ['red', 'blue']
for class_value in [0, 1]:
    mask = y == class_value
    plt.scatter(X[mask, 0], X[mask, 1], 
                c=colors[class_value], 
                label=f'Class {class_value}',
                alpha=0.6, s=50)

plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('Classification Problem', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 1.3: Train/Test Split

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training: {len(X_train)} samples")
print(f"Test: {len(X_test)} samples")

### 1.4: Train a Small Decision Tree

In [ ]:
# Create shallow tree for visualization
tree_small = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_small.fit(X_train, y_train)

# Evaluate
train_acc = tree_small.score(X_train, y_train)
test_acc = tree_small.score(X_test, y_test)

print("Decision Tree (max_depth=3):")
print(f"  Training accuracy: {train_acc:.3f}")
print(f"  Test accuracy: {test_acc:.3f}")

### 1.5: Visualize the Tree

In [ ]:
# Plot decision tree
plt.figure(figsize=(15, 8))
plot_tree(tree_small, 
          feature_names=['Feature 1', 'Feature 2'],
          class_names=['Class 0', 'Class 1'],
          filled=True,
          rounded=True,
          fontsize=10)
plt.title('Decision Tree Structure', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 How to read this:")
print("  - Top box: root node (first question)")
print("  - Each box: a decision point")
print("  - Color: class prediction (orange=class 0, blue=class 1)")
print("  - Samples: number of training examples")

### 1.6: Visualize Decision Boundary

In [ ]:
# Create mesh for decision boundary
h = 0.02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

print("Creating decision boundary visualization...")
print("(This creates a grid and predicts each point)")

In [ ]:
# Make predictions on mesh
Z = tree_small.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot decision boundary
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap='RdBu')
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, 
            cmap='RdBu', edgecolor='black', s=50)
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('Decision Tree: Decision Boundary', fontsize=14, fontweight='bold')
plt.colorbar(label='Predicted Class')
plt.tight_layout()
plt.show()

print("💡 Notice the rectangular decision boundaries!")
print("   Trees split space with axis-aligned cuts.")

---

## Part 2: The Overfitting Problem

**Trees can grow too deep** → memorize training data!

### 2.1: Train a Deep Tree

In [ ]:
# Train tree with no depth limit
tree_deep = DecisionTreeClassifier(random_state=42)
tree_deep.fit(X_train, y_train)

# Evaluate
deep_train = tree_deep.score(X_train, y_train)
deep_test = tree_deep.score(X_test, y_test)

print("Deep Decision Tree (no limit):")
print(f"  Training accuracy: {deep_train:.3f}")
print(f"  Test accuracy: {deep_test:.3f}")
print(f"  Tree depth: {tree_deep.get_depth()}")
print(f"  Number of leaves: {tree_deep.get_n_leaves()}")

if deep_train - deep_test > 0.1:
    print("\n⚠️ OVERFITTING: Perfect on training, worse on test!")

### 2.2: Compare Different Depths

In [ ]:
# Try different max_depth values
depths = range(1, 15)
train_scores = []
test_scores = []

for depth in depths:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree.fit(X_train, y_train)
    train_scores.append(tree.score(X_train, y_train))
    test_scores.append(tree.score(X_test, y_test))

print("Tested depths from 1 to 14")
print(f"Best test score: {max(test_scores):.3f} at depth {test_scores.index(max(test_scores))+1}")

### 2.3: Visualize Overfitting

In [ ]:
# Plot training vs test scores
plt.figure(figsize=(10, 6))
plt.plot(depths, train_scores, 'o-', label='Training', linewidth=2)
plt.plot(depths, test_scores, 's-', label='Test', linewidth=2)
plt.xlabel('Max Depth', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Decision Tree: Depth vs Performance', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Key insights:")
print("  - Shallow tree: underfits (both scores low)")
print("  - Medium depth: good balance")
print("  - Deep tree: overfits (train high, test drops)")

---

## Part 3: Random Forest (Ensemble Power!)

**Idea:** Train many trees, average their predictions

**How:** Each tree sees different random subset of data + features

**Result:** More accurate + less overfitting!

### 3.1: Train Random Forest

In [ ]:
# Create Random Forest
rf = RandomForestClassifier(
    n_estimators=100,  # number of trees
    max_depth=10,
    random_state=42
)

rf.fit(X_train, y_train)

print("✅ Random Forest trained!")
print(f"   Number of trees: {rf.n_estimators}")

### 3.2: Evaluate Random Forest

In [ ]:
# Compare single tree vs forest
rf_train = rf.score(X_train, y_train)
rf_test = rf.score(X_test, y_test)

print("Performance Comparison:")
print("\nSingle Deep Tree:")
print(f"  Training: {deep_train:.3f}")
print(f"  Test:     {deep_test:.3f}")
print(f"  Gap:      {deep_train - deep_test:.3f}")

print("\nRandom Forest:")
print(f"  Training: {rf_train:.3f}")
print(f"  Test:     {rf_test:.3f}")
print(f"  Gap:      {rf_train - rf_test:.3f}")

if rf_test > deep_test:
    print("\n✅ Random Forest performs better!")
    print("   Less overfitting, better generalization!")

### 3.3: Effect of Number of Trees

In [ ]:
# Try different numbers of trees
n_trees = [1, 5, 10, 25, 50, 100, 200]
train_scores_rf = []
test_scores_rf = []

for n in n_trees:
    rf_temp = RandomForestClassifier(n_estimators=n, random_state=42)
    rf_temp.fit(X_train, y_train)
    train_scores_rf.append(rf_temp.score(X_train, y_train))
    test_scores_rf.append(rf_temp.score(X_test, y_test))

print("Tested different forest sizes")
print(f"Best with {n_trees[test_scores_rf.index(max(test_scores_rf))]} trees")

### 3.4: Visualize Forest Growth

In [ ]:
# Plot performance vs number of trees
plt.figure(figsize=(10, 6))
plt.plot(n_trees, train_scores_rf, 'o-', label='Training', linewidth=2)
plt.plot(n_trees, test_scores_rf, 's-', label='Test', linewidth=2)
plt.xlabel('Number of Trees', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Random Forest: Effect of Forest Size', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 More trees usually help, but diminishing returns!")

### 3.5: Random Forest Decision Boundary

In [ ]:
# Predict on mesh
Z_rf = rf.predict(np.c_[xx.ravel(), yy.ravel()])
Z_rf = Z_rf.reshape(xx.shape)

# Plot
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z_rf, alpha=0.3, cmap='RdBu')
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, 
            cmap='RdBu', edgecolor='black', s=50)
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('Random Forest: Decision Boundary', fontsize=14, fontweight='bold')
plt.colorbar(label='Predicted Class')
plt.tight_layout()
plt.show()

print("💡 Smoother boundary than single tree!")

---

## Part 4: Feature Importance

**Question:** Which features matter most?

**Trees tell us!** Based on how much each feature improves splits

### 4.1: Create Dataset with Real Features

In [ ]:
# Load diabetes dataset for regression
diabetes = load_diabetes()
X_diab = diabetes.data
y_diab = diabetes.target
feature_names = diabetes.feature_names

print(f"Diabetes dataset: {len(X_diab)} patients")
print(f"Features: {feature_names}")
print(f"Target: Disease progression (continuous)")

### 4.2: Train Random Forest Regressor

In [ ]:
# Split data
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_diab, y_diab, test_size=0.2, random_state=42
)

# Train Random Forest for regression
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(X_train_d, y_train_d)

# Evaluate
r2 = rf_reg.score(X_test_d, y_test_d)
print(f"✅ Random Forest Regressor trained!")
print(f"   Test R² Score: {r2:.3f}")

### 4.3: Extract Feature Importances

In [ ]:
# Get importances
importances = rf_reg.feature_importances_

# Create DataFrame
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("\nFeature Importances:")
print(importance_df.to_string(index=False))

### 4.4: Visualize Feature Importance

In [ ]:
# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'], color='skyblue')
plt.xlabel('Importance', fontsize=12)
plt.title('Random Forest: Feature Importances', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print(f"\n💡 Most important feature: {importance_df.iloc[0]['feature']}")
print(f"   Contributes {importance_df.iloc[0]['importance']:.1%} to predictions")

---

*[Continuing with Gradient Boosting in next section...]*